In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
import re
import datetime

from packages.parquet_cached_df import ParquetCachedDF

In [2]:
garmin_df = pd.read_csv("./weight_loss_dfs/from_garmin_weight.csv")
date_re = re.compile(r"(?: +)?(?P<month>[a-zA-Z]{3}) (?P<day>\d{1,2}), (?P<year>\d{4})")
time_re = re.compile(r"(?P<hour>\d{1,2}):(?P<minute>\d{2}) (?P<am_pm>[AP]M)")

In [3]:
display(garmin_df)

,Time,Weight,Change,BMI,Body Fat,Skeletal Muscle Mass,Bone Mass,Body Water
0,"Aug 24, 2021",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6:30 AM,174.0 lbs,0.4 lbs,23.6,21 %,73.7 lbs,11.1 lbs,57.7 %
2,"Aug 23, 2021",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6:05 AM,174.4 lbs,0.7 lbs,23.6,21.7 %,73.7 lbs,11.0 lbs,57.1 %
4,"Aug 22, 2021",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5:59 AM,175.1 lbs,1.9 lbs,23.7,22.2 %,73.9 lbs,11.0 lbs,56.8 %
6,"Aug 19, 2021",NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6:15 AM,177.0 lbs,4.0 lbs,24.0,22 %,74.4 lbs,11.2 lbs,56.9 %
8,"Aug 18, 2021",NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,6:02 AM,173.0 lbs,0.9 lbs,23.4,21.3 %,73.4 lbs,10.9 lbs,57.4 %


In [15]:
pertinent_cols = ["Weight", "BMI", "Body Fat", "Skeletal Muscle Mass", "Bone Mass", "Body Water"]

clean_funcs = {
    "Weight": lambda x: float(x.split()[0]),
    "BMI": lambda x: float(x),
    "Body Fat": lambda x: float(x.split()[0]), 
    "Skeletal Muscle Mass": lambda x: float(x.split()[0]), 
    "Bone Mass": lambda x: float(x.split()[0]), 
    "Body Water": lambda x: float(x.split()[0]), 
}

row_headers = {
    "Weight": "Weight (lb)",
    "BMI": "BMI",
    "Body Fat": "Body Fat (%)",
    "Skeletal Muscle Mass": "Skeletal Muscle Mass (lb)",
    "Bone Mass": "Bone Mass (lb)",
    "Body Water": "Body Water (%)",
}


cleaned = {row_headers[col]: [] for col in pertinent_cols}
cleaned["date"] = []
date = None

def add_row(di, row, date, replace_last=False):
    
    global row_headers, clean_funcs, pertinent_cols
    
    if replace_last:
        
        assert date == di["date"][-1], "invalid date for replace"
        
        for col in pertinent_cols:
            if row[col] == "--":
                di[row_headers[col]] = np.nan
            else:
                di[row_headers[col]][-1] = clean_funcs[col](row[col])
    else:
        cleaned["date"].append(date)

        for col in pertinent_cols:
            if row[col] == "--":
                di[row_headers[col]].append(np.nan)
            else:
                di[row_headers[col]].append(clean_funcs[col](row[col]))

for i, row in garmin_df.iterrows():
    date_match = date_re.match(row["Time"])
    time_match = time_re.match(row["Time"])
    if date_match:
        
        _d = date_match.groupdict()
        
        date = datetime.datetime.strptime(
            f"{_d['year']}-{_d['month']}-{_d['day']}",
            "%Y-%b-%d"
        ).strftime("%Y-%m-%d")
        
    elif time_match:
        assert date, "no date!?"
        if cleaned["date"]: 
            if date == cleaned["date"][-1]:
                add_row(di=cleaned, row=row, date=date, replace_last=True)
            else:
                add_row(di=cleaned, row=row, date=date, replace_last=False)
        else:
            add_row(di=cleaned, row=row, date=date, replace_last=False)
    else:
        raise ValueError(f"row after {date} doesn't match!")
 

In [16]:
df_garmin_cleaned = pd.DataFrame(cleaned).set_index("date")

In [17]:
df_path = "./weight_loss_dfs/jordan_df_add_garmin.pqt"
df_manager = ParquetCachedDF(file_path=df_path)

In [18]:
df_personal = df_manager.get_df()
display(df_personal)

,Weight (lb),Body Fat (%),BMI,Skeletal Muscle Mass (lb),Bone Mass (lb),Body Water (%),Waist (in),Belly (in),Hips (in),Chest (in),Bicep (in),Thigh (in),Calf (in),Target Calories (kcal),Consumed Calories (kcal),Active Calories (kcal),Resting Heart Rate (bpm),Workout,Cardio,Stretch,Meditate,Mile Time (min),Mode
2021-08-24,174.0,21.0,23.6,73.7,11.1,57.7,32.7500,35.5000,39.7500,37.5625,12.2500,21.5625,13.9375,1728.0,NaN,NaN,NaN,False,False,False,False,NaN,Cutting
2021-08-23,174.4,21.7,23.6,73.7,11.0,57.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1728.0,2066.0,494.0,60.0,False,True,True,False,NaN,Cutting
2021-08-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1728.0,NaN,NaN,48.0,False,False,False,False,NaN,Cutting
2021-05-21,174.2,22.3,23.6,73.7,10.8,56.7,33.2500,35.3750,39.1875,37.8750,12.5000,21.9375,13.7500,1728.0,NaN,NaN,NaN,False,False,False,False,NaN,Cutting
2021-05-20,175.5,22.0,23.8,74.0,11.0,56.9,33.5000,35.6250,39.6875,38.0625,12.5625,22.0000,13.9375,1728.0,2211.0,699.0,57.0,False,True,True,False,NaN,Cutting
2021-05-19,178.2,21.6,24.1,74.7,11.3,57.2,33.8750,35.8750,40.0000,38.5000,12.5625,22.1250,14.0000,1728.0,2040.0,665.0,58.0,True,False,True,False,NaN,Cutting
2021-05-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1728.0,2140.0,544.0,61.0,False,True,True,False,NaN,Cutting
2021-05-17,177.5,21.4,24.0,74.5,11.3,57.4,33.3750,35.8750,39.7500,38.2500,12.5000,22.1250,14.0000,1728.0,NaN,NaN,53.0,True,False,True,False,NaN,Cutting
2021-05-16,177.7,22.4,24.1,74.6,11.1,56.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1728.0,NaN,906.0,60.0,False,True,False,False,NaN,Cutting
2021-05-15,176.2,21.8,23.9,74.2,11.1,57.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1728.0,NaN,406.0,50.0,False,False,True,False,NaN,Cutting


In [56]:
print(df_personal.shape)
print(df_garmin_cleaned.shape)

(78, 23)
(121, 6)


In [57]:
df_merged = df_personal.merge(df_garmin_cleaned, how="outer", suffixes=(" MINE", " GARMIN"), left_index=True, right_index=True)
df_merged.columns

Index(['Weight (lb) MINE', 'Body Fat (%) MINE', 'BMI MINE',
       'Skeletal Muscle Mass (lb) MINE', 'Bone Mass (lb) MINE',
       'Body Water (%) MINE', 'Waist (in)', 'Belly (in)', 'Hips (in)',
       'Chest (in)', 'Bicep (in)', 'Thigh (in)', 'Calf (in)',
       'Target Calories (kcal)', 'Consumed Calories (kcal)',
       'Active Calories (kcal)', 'Resting Heart Rate (bpm)', 'Workout',
       'Cardio', 'Stretch', 'Meditate', 'Mile Time (min)', 'Mode',
       'Weight (lb) GARMIN', 'BMI GARMIN', 'Body Fat (%) GARMIN',
       'Skeletal Muscle Mass (lb) GARMIN', 'Bone Mass (lb) GARMIN',
       'Body Water (%) GARMIN'],
      dtype='object')

In [58]:
mine_re = re.compile(r"(?P<col>.*) MINE$")
garmin_re = re.compile(r".*GARMIN$")
keep_cols = []
cols = df_merged.columns
print(cols)
for col in cols:
    if mine_re.match(col):
        column_header = mine_re.match(col).groupdict()["col"]
        print(column_header)
        df_merged[column_header] = df_merged[f"{column_header} GARMIN"]
        keep_cols.append(column_header)
    elif garmin_re.match(col):
        print("garmin")
        pass
    else:
        keep_cols.append(col)
print(keep_cols)
#
#df_merged[
#    (df_merged["Weight (lb) MINE"] != df_merged["Weight (lb) GARMIN"])
#    | (df_merged["Body Fat (%) MINE"] != df_merged["Body Fat (%) GARMIN"])
#    | (df_merged["BMI MINE"] != df_merged["BMI GARMIN"])
#    | (df_merged["Skeletal Muscle Mass (lb) MINE"] != df_merged["Skeletal Muscle Mass (lb) GARMIN"])
#    | (df_merged["Bone Mass (lb) MINE"] != df_merged["Bone Mass (lb) GARMIN"])
#    | (df_merged["Body Water (%) MINE"] != df_merged["Body Water (%) GARMIN"])
#    
#][[
#    "Weight (lb) MINE", "BMI MINE", "Body Fat (%) MINE", "Skeletal Muscle Mass (lb) MINE", "Bone Mass (lb) MINE","Body Water (%) MINE",
#    "Weight (lb) GARMIN", "BMI GARMIN", "Body Fat (%) GARMIN","Skeletal Muscle Mass (lb) GARMIN", "Bone Mass (lb) GARMIN","Body Water (%) GARMIN",
#]]

Index(['Weight (lb) MINE', 'Body Fat (%) MINE', 'BMI MINE',
       'Skeletal Muscle Mass (lb) MINE', 'Bone Mass (lb) MINE',
       'Body Water (%) MINE', 'Waist (in)', 'Belly (in)', 'Hips (in)',
       'Chest (in)', 'Bicep (in)', 'Thigh (in)', 'Calf (in)',
       'Target Calories (kcal)', 'Consumed Calories (kcal)',
       'Active Calories (kcal)', 'Resting Heart Rate (bpm)', 'Workout',
       'Cardio', 'Stretch', 'Meditate', 'Mile Time (min)', 'Mode',
       'Weight (lb) GARMIN', 'BMI GARMIN', 'Body Fat (%) GARMIN',
       'Skeletal Muscle Mass (lb) GARMIN', 'Bone Mass (lb) GARMIN',
       'Body Water (%) GARMIN'],
      dtype='object')
Weight (lb)
Body Fat (%)
BMI
Skeletal Muscle Mass (lb)
Bone Mass (lb)
Body Water (%)
garmin
garmin
garmin
garmin
garmin
garmin
['Weight (lb)', 'Body Fat (%)', 'BMI', 'Skeletal Muscle Mass (lb)', 'Bone Mass (lb)', 'Body Water (%)', 'Waist (in)', 'Belly (in)', 'Hips (in)', 'Chest (in)', 'Bicep (in)', 'Thigh (in)', 'Calf (in)', 'Target Calories (kcal)', '

In [59]:
display(df_merged[keep_cols])

,Weight (lb),Body Fat (%),BMI,Skeletal Muscle Mass (lb),Bone Mass (lb),Body Water (%),Waist (in),Belly (in),Hips (in),Chest (in),Bicep (in),Thigh (in),Calf (in),Target Calories (kcal),Consumed Calories (kcal),Active Calories (kcal),Resting Heart Rate (bpm),Workout,Cardio,Stretch,Meditate,Mile Time (min),Mode
2020-12-23,NaN,NaN,NaN,NaN,NaN,NaN,34.0000,37.3750,41.0000,38.2500,12.6250,20.6250,NaN,1790.0,NaN,0.0,NaN,True,False,None,False,NaN,Cutting
2020-12-24,NaN,NaN,NaN,NaN,NaN,NaN,34.7500,37.1250,40.6250,38.1250,12.2500,21.5000,NaN,1790.0,2184.0,0.0,NaN,False,True,None,False,NaN,Cutting
2020-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1740.0,2239.0,0.0,NaN,False,False,None,False,NaN,Cutting
2020-12-26,NaN,NaN,NaN,NaN,NaN,NaN,33.1250,36.7500,41.1250,38.2500,13.0000,21.7500,NaN,1740.0,NaN,0.0,71.0,True,False,None,False,NaN,Cutting
2020-12-27,NaN,NaN,NaN,NaN,NaN,NaN,32.6250,36.1250,40.3750,38.7500,12.8750,21.5000,NaN,1740.0,1728.0,0.0,67.0,False,True,None,False,10.100000,Cutting
2020-12-28,NaN,NaN,NaN,NaN,NaN,NaN,33.6250,35.7500,40.2500,38.2500,12.5000,21.7500,NaN,1740.0,1764.0,0.0,66.0,False,False,None,False,NaN,Cutting
2020-12-29,NaN,NaN,NaN,NaN,NaN,NaN,33.0000,35.7500,40.2500,37.9375,12.4375,22.1250,NaN,1740.0,2204.0,0.0,NaN,False,False,True,False,NaN,Cutting
2020-12-30,NaN,NaN,NaN,NaN,NaN,NaN,32.5000,36.6250,40.0000,37.6875,12.6250,22.2500,14.1250,1740.0,NaN,0.0,NaN,True,False,False,False,NaN,Cutting
2020-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1740.0,NaN,0.0,NaN,False,True,False,False,NaN,Cutting
2021-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1740.0,NaN,0.0,NaN,False,False,False,False,NaN,Cutting


In [61]:
df = df_merged[
    [
        'Weight (lb)',
        'Body Fat (%)',
        'BMI',
        'Skeletal Muscle Mass (lb)',
        'Bone Mass (lb)',
        'Body Water (%)',
        'Waist (in)',
        'Belly (in)',
        'Hips (in)',
        'Chest (in)',
        'Bicep (in)',
        'Thigh (in)',
        'Calf (in)',
        'Target Calories (kcal)',
        'Consumed Calories (kcal)',
        'Active Calories (kcal)',
        'Resting Heart Rate (bpm)',
        'Workout',
        'Cardio',
        'Stretch',
        'Meditate',
        'Mile Time (min)',
        'Mode'
    ] 
]

In [66]:
df = df.sort_index(ascending=False)
df["Mode"] = "Cutting"

In [68]:
df_path = "./weight_loss_dfs/jordan_df.pqt"
df_manager.set_df(df)
df_manager.save_df(file_path=df_path)
